# IS4242 Group Assignment Part 2
**November 11, 2020**

#### Name: LECK WEI SHENG IAN
#### NUS ID: A0168177R
#### Name: WOO KENG THONG
#### NUS ID: A0167991L

Note the following features for each pump:
- Total Static Head (TSH)
- Total vertical distance that a pump raises water against gravity
- Water available to a water point
- Local Government Area (LGA)
- Geographic Location
- Population served
- Latitude and Longitude - GPS coordinates
- Labels: functional, non-functional and functional-needs-repair

Your team is a consultant to the government of Tanzania, who has asked you to determine, for each LGA:
which non-functional pumps can be replaced at minimum possible cost in order to serve water to everyone 
You should develop a solution where your client can input the name of the LGA and will receive the following outputs:
- the names of the non-functional pumps to be replaced 
- the total cost

#### Use latitude and longitude for distance computation

# Instructions:
- Restart and run all cells under Kernel
- Input desired lga to check. Note that only valid lga will be allowed. 
### Output will be at the bottom of the notebook
### Invalid lga will not result in any results.

Input desired lga to be checked (E.g. Ilemela)

In [1]:
lga_check = input("Enter desired lga: ") 

Enter desired lga: Ilemela


## Preprocessing

In [2]:
import pyomo.environ as pe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Read training and test data that has not been preprocessed.

In [3]:
labels = pd.read_csv('training-set-labels.csv')
df = pd.read_csv('training-set-values.csv')
test_df = pd.read_csv('test-set-values.csv')
test_labels = pd.read_csv('submission_rf_ax.csv')

df = pd.merge(df, labels, on='id')
test_df = pd.merge(test_df, test_labels, on='id')

We will be processing un-processed data first in order to select for the following criteria:
- Don't have location information
- Have pumps with 0 surrounding population
- Have no name ('none') for the pump
- Are functional-needs-repair pumps
- Date_recorded before 2014
<Br>

This data will subsequently be merged with pre-processed data from part1 in order to obtain cleaner data - data without any missing values.

Check if any of the pumps' status were recorded after 2013 

In [4]:
df['date_recorded'] = pd.to_datetime(df['date_recorded'])
print('train data after 2013 = ', len(df[df.date_recorded.dt.year>2013]))

test_df['date_recorded'] = pd.to_datetime(test_df['date_recorded'])
print('test data after 2013 = ', len(test_df[test_df.date_recorded.dt.year>2013]))

train data after 2013 =  0
test data after 2013 =  0


Check population data

In [5]:
print('df population')
print(df.population.value_counts())
print('test df population')
print(test_df.population.value_counts())

df population
0       21381
1        7025
200      1940
150      1892
250      1681
        ...  
3241        1
1960        1
1685        1
2248        1
1439        1
Name: population, Length: 1049, dtype: int64
test df population
0       5453
1       1757
150      436
200      430
250      406
        ... 
244        1
252        1
284        1
2365       1
7000       1
Name: population, Length: 637, dtype: int64


Create `to_remove` and `test_to_remove`, which will track row ids to be removed from clean data subsequently.<br>
Obtain row ids with `population = 0`, and remove from `df` and `test_df`

In [6]:
to_remove = df.loc[df['population'] == 0]
df = df.loc[df['population'] != 0]
print('df to remove', to_remove.shape)

# Repeat for test data
test_to_remove = test_df.loc[test_df['population'] == 0]
test_df = test_df.loc[test_df['population'] != 0]
print('test df to remove', test_to_remove.shape)

df to remove (21381, 41)
test df to remove (5453, 41)


In [7]:
print('remaining df', df.shape)
print('remaining test df', test_df.shape)

remaining df (38019, 41)
remaining test df (9397, 41)


Check pump name data

In [8]:
print('df wpt name')
print(df.wpt_name.value_counts())
print('test df wpt name')
print(test_df.wpt_name.value_counts())

df wpt name
none                 2248
Shuleni              1251
Zahanati              524
Msikitini             456
Kanisani              217
                     ... 
Kwa Bakari Mwenye       1
Kwa Mzee Dinongo        1
Amira                   1
Kwa Mbigi               1
Kumureka                1
Name: wpt_name, Length: 24849, dtype: int64
test df wpt name
none             552
Shuleni          306
Zahanati         132
Msikitini         93
Sokoni            56
                ... 
New Kisoyi         1
Kwambunji          1
Haji Yusuph        1
Kwa Ndilaliha      1
Kwa Dodo           1
Name: wpt_name, Length: 7072, dtype: int64


Obtain row ids with `wpt_name = none`, and remove from `df` and `test_df`

In [9]:
to_remove = to_remove.append(df.loc[df['wpt_name'] == 'none'])
df = df.loc[df['wpt_name'] != 'none']
print('df to remove', to_remove.shape)

# Repeat for test data
test_to_remove = test_to_remove.append(test_df.loc[test_df['wpt_name'] == 'none'])
test_df = test_df.loc[test_df['wpt_name'] != 'none']
print('test df to remove', test_to_remove.shape)

df to remove (23629, 41)
test df to remove (6005, 41)


In [10]:
print('remaining df', df.shape)
print('remaining test df', test_df.shape)

remaining df (35771, 41)
remaining test df (8845, 41)


Check status group

In [11]:
print('df status group')
print(df.status_group.value_counts())
print('test df status group')
print(test_df.status_group.value_counts())

df status group
functional                 19357
non functional             13925
functional needs repair     2489
Name: status_group, dtype: int64
test df status group
functional        5197
non functional    3648
Name: status_group, dtype: int64


Obtain row ids with `status group = functional needs repair`, and remove from `df`. There is no need to remove from `test_df` as there are no rows that fit this filter

In [12]:
to_remove = to_remove.append(df.loc[df['status_group'] == 'functional needs repair'])
df = df.loc[df['status_group'] != 'functional needs repair']
print('df to remove', to_remove.shape)

df to remove (26118, 41)


In [13]:
print('remaining df', df.shape)
print('remaining test df', test_df.shape)

remaining df (33282, 41)
remaining test df (8845, 41)


Check for missing data in locations

In [14]:
print('df')
print(df.isnull().sum())
print('test df')
print(test_df.isnull().sum())

df
id                           0
amount_tsh                   0
date_recorded                0
funder                    1383
gps_height                   0
installer                 1388
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                   7
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            2227
recorded_by                  0
scheme_management         2288
scheme_name              14023
permit                    1805
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quali

There is no missing data for locations predictors, but I will also check whether the latlong values are accurate.<br>
Tanzania's latlong are generally within -1° to -12° latitude, and 29° to 41° longitude. I will do a check for erroneous latlong data as well.

In [15]:
print('df latitude >-1 = ', len(df[df.latitude>-1]))
print('df latitude <-12 = ', len(df[df.latitude<-12]))
print('df longitude <29 = ', len(df[df.longitude<29]))
print('df longitude >41 = ', len(df[df.longitude>41]))

print('test df latitude >-1 = ', len(test_df[test_df.latitude>-1]))
print('test df latitude <-12 = ', len(test_df[test_df.latitude<-12]))
print('test df longitude <29 = ', len(test_df[test_df.longitude<29]))
print('test df longitude >41 = ', len(test_df[test_df.longitude>41]))

df latitude >-1 =  0
df latitude <-12 =  0
df longitude <29 =  0
df longitude >41 =  0
test df latitude >-1 =  0
test df latitude <-12 =  0
test df longitude <29 =  0
test df longitude >41 =  0


#### Import cleaned data from part1, and remove the relevant rows them with `to_remove` and `test_to_remove`, having finished preprocessing for part2

In [16]:
df = pd.read_csv('clean.csv')
test_df = pd.read_csv('clean_test.csv')
test_labels = pd.read_csv('submission_rf_ax.csv')

test_df = pd.merge(test_df, test_labels, on='id')

In [17]:
df = df[~df['id'].isin(to_remove['id'])]
test_df = test_df[~test_df['id'].isin(test_to_remove['id'])]

In [18]:
print('remaining df', df.shape)
print('remaining test df', test_df.shape)

remaining df (33282, 28)
remaining test df (8845, 28)


Merge `df` and `test_df` to use data from both training and test for part 2

In [19]:
total_df = pd.concat([df,test_df])
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42127 entries, 1 to 14849
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     42127 non-null  int64  
 1   amount_tsh             42127 non-null  float64
 2   funder                 41786 non-null  object 
 3   gps_height             42127 non-null  float64
 4   installer              41780 non-null  object 
 5   longitude              42127 non-null  float64
 6   latitude               42127 non-null  float64
 7   wpt_name               42127 non-null  object 
 8   basin                  42127 non-null  object 
 9   region                 42127 non-null  object 
 10  district_code          42127 non-null  int64  
 11  lga                    42127 non-null  object 
 12  ward                   42127 non-null  object 
 13  population             42127 non-null  float64
 14  public_meeting         42127 non-null  float64
 15  re

#### Select rows based on `lga` input from user

In [20]:
lga_df = total_df.loc[total_df['lga'] == lga_check]

In [21]:
lga_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 2457 to 10949
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     27 non-null     int64  
 1   amount_tsh             27 non-null     float64
 2   funder                 27 non-null     object 
 3   gps_height             27 non-null     float64
 4   installer              27 non-null     object 
 5   longitude              27 non-null     float64
 6   latitude               27 non-null     float64
 7   wpt_name               27 non-null     object 
 8   basin                  27 non-null     object 
 9   region                 27 non-null     object 
 10  district_code          27 non-null     int64  
 11  lga                    27 non-null     object 
 12  ward                   27 non-null     object 
 13  population             27 non-null     float64
 14  public_meeting         27 non-null     float64
 15  re

Split the df by `functional` and `non functional` pumps. Functional pumps are not limited by `lga`, as `non functional` pumps can be closest to `functional` pumps outside of `lga`

In [22]:
funcdf = total_df.loc[total_df['status_group'] == 'functional']
nonfuncdf = lga_df.loc[lga_df['status_group'] == 'non functional']
funcdf.reset_index(drop=True,inplace=True)
nonfuncdf.reset_index(drop=True,inplace=True)
print(funcdf.shape)
print(nonfuncdf.shape)

(24554, 28)
(15, 28)


I will be using the haversine formula to get the Euclidean distance between two pumps obtained from <br>
https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points

In [23]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6372.8 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

Create a dictionary to get closest distance to `functional` pump from `non functional` pump.<br>
Create `remaining_population` column to track remaining population that each `functional` pump can support. It is the existing community size as each `functional` pump can serve a population twice its community size.<br>

In [24]:
closest = []
remaining_pop = funcdf['population']
funcdf = funcdf.assign(remaining_pop = remaining_pop)
funcdf

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,basin,region,...,extraction_type_class,management,payment_type,water_quality,quantity_group,source,waterpoint_type,status_group,operational_years,remaining_pop
0,8776,374.652174,Grumeti,1399.000000,GRUMETI,34.698766,-2.147466,Zahanati,Lake Victoria,Mara,...,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,functional,3.0,280.0
1,34310,25.000000,Lottery Club,686.000000,World vision,37.460664,-3.821329,Kwa Mahundi,Pangani,Manyara,...,gravity,vwc,per bucket,soft,enough,dam,communal standpipe multiple,functional,4.0,250.0
2,9944,20.000000,Mkinga Distric Coun,675.978008,DWE,39.172796,-4.765587,Tajiri,Pangani,Tanga,...,submersible,vwc,per bucket,salty,enough,other,communal standpipe multiple,functional,2.0,1.0
3,49056,43.979592,Private,62.000000,Private,39.209518,-7.034139,Mzee Hokororo,Wami / Ruvu,Pwani,...,submersible,private operator,never pay,salty,enough,machine dbh,other,functional,0.0,345.0
4,50409,200.000000,Danida,1062.000000,DANIDA,35.770258,-10.574175,Kwa Alid Nchimbi,Lake Nyasa,Ruvuma,...,handpump,wug,on failure,soft,insufficient,shallow well,hand pump,functional,26.0,250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24549,57316,432.590909,Devon Aid Korogwe,347.000000,Local technician,38.613415,-4.815222,Jahiya,Pangani,Tanga,...,gravity,other,never pay,soft,insufficient,spring,communal standpipe,functional,6.0,300.0
24550,65541,632.459677,Oxfarm,1641.000000,OXFARM,29.768139,-4.480618,Mwandami,Lake Tanganyika,Kigoma,...,other,vwc,never pay,soft,enough,spring,other,functional,18.0,1400.0
24551,18990,1000.000000,Hiap,679.918735,HIAP,37.451633,-5.350428,Bonde La Mkondoa,Pangani,Tanga,...,handpump,vwc,annually,salty,insufficient,shallow well,hand pump,functional,17.0,2960.0
24552,28749,1440.961538,NaN,1476.000000,NaN,34.739804,-4.585587,Bwawani,Internal,Singida,...,gravity,vwc,never pay,soft,insufficient,dam,communal standpipe,functional,3.0,200.0


Iterate through `non functional` pumps and `functional` pumps. <br>
Check if the `functional` pump's `remaining_population` >= `non functional` pump's `population` to ensure that it can support the additional `population`.<br>
If it can, calculate distance between `functional` and `non functional` pump, and check if it is the nearest pump.<br>
Outside each inner loop, the nearest pump will be recorded in the `closest` dictionary.<br>
Also reduce `remaining_population` by the additional amount of `population` supported.
#### Take note that this algorithm works on a first-come-first-served basis. For example, if there are multiple `non functional` pumps A, B, C that are nearest to `functional` pump D but pump D can only support 2 additional pumps due to population constraint despite all 3 having the same nearest distance, pumps A and B will be supported by pump D. Pump C will have then have to find another nearest pump.

In [25]:
for i, nonfunc_pump in nonfuncdf.iterrows():
    
    nonfunc_lat = nonfunc_pump.latitude
    nonfunc_lon = nonfunc_pump.longitude
    nonfunc_pop = nonfunc_pump.population
    d = None
    # To track index of nearest pump
    pump_idx = None
    
    for j, func_pump in funcdf.iterrows():
        func_lat = func_pump.latitude
        func_lon = func_pump.longitude
        func_rem_pop = func_pump.remaining_pop
        
        if(func_rem_pop>=nonfunc_pop):
            temp_d = haversine(nonfunc_lon,nonfunc_lat, func_lon, func_lat)
            if(d == None or d > temp_d):
                d = temp_d
                pump_idx = j
            
    closest.append(d)
    funcdf.at[pump_idx,'remaining_pop'] = funcdf.at[pump_idx,'remaining_pop']-nonfunc_pop
    print('currently at iteration: ',i)

currently at iteration:  0
currently at iteration:  1
currently at iteration:  2
currently at iteration:  3
currently at iteration:  4
currently at iteration:  5
currently at iteration:  6
currently at iteration:  7
currently at iteration:  8
currently at iteration:  9
currently at iteration:  10
currently at iteration:  11
currently at iteration:  12
currently at iteration:  13
currently at iteration:  14


Add `closest_distance` column to `nonfuncdf` to record the nearest distance of each `non functional` pump to the next nearest `functional` pump that can still support additional `population`.<br>

In [26]:
nonfuncdf = nonfuncdf.assign(closest_dist = closest)
nonfuncdf

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,basin,region,...,extraction_type_class,management,payment_type,water_quality,quantity_group,source,waterpoint_type,status_group,operational_years,closest_dist
0,73178,1913.636364,L,1195.0,L,32.956523,-2.494353,Kwa Julias Mabula,Lake Victoria,Mwanza,...,other,vwc,never pay,salty,enough,shallow well,other,non functional,45.0,3.315638
1,7415,1913.636364,H,1230.0,H,32.977191,-2.516619,Kwa Buswelu B,Lake Victoria,Mwanza,...,other,vwc,never pay,soft,enough,shallow well,other,non functional,25.0,0.173923
2,38707,1913.636364,D,1244.0,D,32.972719,-2.528716,Kwa Bulola A,Lake Victoria,Mwanza,...,handpump,vwc,never pay,salty,insufficient,shallow well,hand pump,non functional,10.0,1.603844
3,53088,1913.636364,H,1226.0,H,32.955597,-2.501627,Kwa Chuga,Lake Victoria,Mwanza,...,other,vwc,never pay,soft,enough,shallow well,other,non functional,18.0,3.783605
4,44373,500.000000,H,1246.0,H,32.996093,-2.516047,Kwa Zuberi,Lake Victoria,Mwanza,...,handpump,vwc,on failure,soft,insufficient,shallow well,hand pump,non functional,11.0,1.864577
5,46634,1000.000000,H,1259.0,H,32.984790,-2.496459,Kwa Lwaho Ngogashili,Lake Victoria,Mwanza,...,handpump,vwc,monthly,soft,enough,shallow well,hand pump,non functional,22.0,0.586364
6,35072,1913.636364,H,1225.0,H,32.965734,-2.504294,Kwa Bujingwa,Lake Victoria,Mwanza,...,handpump,vwc,never pay,soft,insufficient,shallow well,hand pump,non functional,25.0,0.291998
7,7568,1913.636364,H,1247.0,H,32.993277,-2.510639,Nyamadoke,Lake Victoria,Mwanza,...,handpump,vwc,never pay,soft,insufficient,shallow well,hand pump,non functional,25.0,1.790540
8,21107,1913.636364,H,1171.0,H,32.924886,-2.465246,Kwa Kabambo,Lake Victoria,Mwanza,...,handpump,vwc,never pay,salty,insufficient,shallow well,hand pump,non functional,20.0,42.464024
9,6416,1000.000000,H,1239.0,H,32.987670,-2.490324,Kwa Jonh Andrew,Lake Victoria,Mwanza,...,other,vwc,monthly,salty,insufficient,shallow well,other,non functional,24.0,44.788389


### Assumptions
- A functional pump can serve twice as many people as its surrounding population in the LGA
- A non-functional pump does not serve anybody 

The LGA population with a non-functional pump can be supplied water by
1. replacing a non-functional pump OR
2. transporting water from the nearest functional pump
<br><br>
Note that
> The cost of replacing a pump is $r_f$ + $r_v TSH$ where $r_f$ = \\$100 is the fixed cost and the variable cost is $r_v$ = \\$0.05 per unit $TSH$ <br>
> The cost of transporting water is $t_f$+$t_v d$ where $t_f$ = \\$100 is the fixed cost and the variable cost is $t_v$ = \\$2000 per unit distance and $d$ is the Euclidean distance between a non-functional pump and the nearest functional pump

## Aim: Minimize overall cost of replacement and transportation
### where

Cost of replacing pump = $r_f$ + $r_v TSH$, $r_f$ = 100, $r_v$ = 0.05 <br>
Cost of transporting water = $t_f$ + $t_v d$, $t_f$ = 100, $t_v$ = 2000

#### Variables:
- $X_i$: Cost of replacing pump i
- $Y_i$: Cost of transporting water to pump i
- $B_i$: Binary value to replace pump i (1) or to consider cost of transporting water (0)

#### Objective:
To minimize replacement cost and transport cost:
$$\sum_{t=start_i}^{end_i} (X_i * B_i) + \sum_{t=start_i}^{end_i} (Y_i *(1-B_i)) $$

The model will choose whether to replace pump i or to transport water from the next nearest pump, depending on which is cheaper. This is illustrated by the equation above. On one hand, $B_i$ will be 1 if the pump is to be replaced, and the cost of transporting water $-$ $Y_i$ will not be taken into account. On the other hand, $B_i$ will be 0 if cost of transporting water is cheaper, and cost of replacing pump $-$ $X_i$ will not be taken into account.

#### Constraints:
Each `functional` pump can support twice its community `population`. This has been accounted for in the preprocessing above.

Create concrete model and obtain the replacement and transport costs for each row.<br>

In [27]:
pump = pe.ConcreteModel(name='Pump Cost Problem')
replacement_costs = []
transportation_costs = []

for i, nonfunc in nonfuncdf.iterrows():
    replacement_costs.append(100 + 0.05*nonfunc.amount_tsh)
    transportation_costs.append(100 + 2000*nonfunc.closest_dist)

Convert `replacement_costs` and `transportation_costs` into dictionary to be called later.

In [28]:
idx = nonfuncdf.id.tolist()
rep_costs = {idx: replacement_costs for idx, replacement_costs in zip(idx, replacement_costs)}
trans_costs = {idx: transportation_costs for idx, transportation_costs in zip(idx, transportation_costs)}

Declare binary variable for model, as well as expression to be used and objective of model

In [29]:
pump.binary = pe.Var(idx, domain=pe.Binary, initialize=0)

def obj_expr(model):
    return sum(rep_costs[i]*pump.binary[i] for i in idx) + sum(trans_costs[i]*(1-pump.binary[i]) for i in idx)

pump.obj = pe.Objective(rule=obj_expr,sense=pe.minimize)
print(pump.obj.expr)

195.68181818181822*binary[73178] + 195.68181818181822*binary[7415] + 195.68181818181822*binary[38707] + 195.68181818181822*binary[53088] + 125.0*binary[44373] + 150.0*binary[46634] + 195.68181818181822*binary[35072] + 195.68181818181822*binary[7568] + 195.68181818181822*binary[21107] + 150.0*binary[6416] + 195.68181818181822*binary[44489] + 195.68181818181822*binary[29971] + 195.68181818181822*binary[73042] + 150.0*binary[17965] + 150.0*binary[3000] + 6731.275766563277*(1 - binary[73178]) + 447.8461848561355*(1 - binary[7415]) + 3307.688513687048*(1 - binary[38707]) + 7667.209803812229*(1 - binary[53088]) + 3829.154801322243*(1 - binary[44373]) + 1272.7274448946284*(1 - binary[46634]) + 683.9964724120398*(1 - binary[35072]) + 3681.080528103045*(1 - binary[7568]) + 85028.04743850106*(1 - binary[21107]) + 89676.77782642025*(1 - binary[6416]) + 89448.41170992632*(1 - binary[44489]) + 223764.78517509124*(1 - binary[29971]) + 88439.29706257307*(1 - binary[73042]) + 105261.61963254609*(1 - b

Use `glpk` solver

In [30]:
opt = pe.SolverFactory('glpk')
results = opt.solve(pump)

In [31]:
pump.display()

Model Pump Cost Problem

  Variables:
    binary : Size=15, Index=binary_index
        Key   : Lower : Value : Upper : Fixed : Stale : Domain
         3000 :     0 :   1.0 :     1 : False : False : Binary
         6416 :     0 :   1.0 :     1 : False : False : Binary
         7415 :     0 :   1.0 :     1 : False : False : Binary
         7568 :     0 :   1.0 :     1 : False : False : Binary
        17965 :     0 :   1.0 :     1 : False : False : Binary
        21107 :     0 :   1.0 :     1 : False : False : Binary
        29971 :     0 :   1.0 :     1 : False : False : Binary
        35072 :     0 :   1.0 :     1 : False : False : Binary
        38707 :     0 :   1.0 :     1 : False : False : Binary
        44373 :     0 :   1.0 :     1 : False : False : Binary
        44489 :     0 :   1.0 :     1 : False : False : Binary
        46634 :     0 :   1.0 :     1 : False : False : Binary
        53088 :     0 :   1.0 :     1 : False : False : Binary
        73042 :     0 :   1.0 :     1 :

## Output

In [32]:
print('Pumps to replace:')
for i in idx:
    if pump.binary[i].value == 1:
        pump_name = nonfuncdf.loc[nonfuncdf['id'] == i].wpt_name.item()
        print('pump name = ', pump_name)

print('Optimum overall cost: $', round(pump.obj.expr(), 2))

Pumps to replace:
pump name =  Kwa Julias Mabula
pump name =  Kwa Buswelu B
pump name =  Kwa Bulola A
pump name =  Kwa Chuga
pump name =  Kwa Zuberi
pump name =  Kwa Lwaho Ngogashili
pump name =  Kwa Bujingwa
pump name =  Nyamadoke
pump name =  Kwa Kabambo
pump name =  Kwa Jonh Andrew
pump name =  Kwa Kaombwe
pump name =  Kwa Lukas
pump name =  Kwa Nyamhongolo
pump name =  Kwa Darajani
pump name =  Kwa Mama Okocha
Optimum overall cost: $ 2681.82
